# TTscan - Define the scan and solver parameters

In this file the usage of TTscan class is demonstrated. TTscan holds the information on what kind of a scan is performed (energy or angle, beam polarization) and some solver parameters. Let's start by doing some imports:

In [1]:
import sys
import os.path

import numpy as np

sys.path.insert(1, '..')

from pyTTE import TTscan, Quantity

TTscan object can be initialized either by passing the parameters of the crystal as keyword arguments, or by reading them from a file. Let's examine the former case first.

The initialization of the class requires at least the following three parameters: _constant_, _scan_, _polarization_. `constant` is a Quantity object either of type energy or angle which determines the type of the scan _i.e._ if the constant is of type energy, then the incident photon energy is kept constant and the rocking angle is scanned, and vice versa.

`scan` is a numpy array wrapped in a Quantity instance of type energy or angle, opposite to that of `constant`. The scan values are given relative to the value corresponding to `constant` via the kinematical Bragg condition. For example, if `constant` $= \theta$, then the zero value of `scan` corresponds to the photon energy $hc/2 d \sin \theta$. 

`polarization` is either `'sigma'` or `'s'` for $\sigma$-polarized beam and `'pi'` or `'p'` for $\pi$-polarized beam.

For example, a energy scan from -100 meV to 100 meV about the kinematical diffraction energy at 87$^\circ$ angle with $\sigma$-polarized beam is defined by:

In [2]:
tts = TTscan(constant = Quantity(87,'deg'), scan=Quantity(np.linspace(-100,100,150),'meV'), polarization = 's')

print(tts)

Scan type     : energy
Scan constant : 87 deg
Polarization  : sigma
Scan points   : 150
Scan range    : manual from -100.0 to 100.0 meV

Integrator                 : zvode_bdf
(Minimum) integration step : 1e-10 um
Alternative starting depth : None



`scan` may also be a positive integer indicating the number of scan points. TakagiTaupin will then automatically calculate the scan limits which may not be perfect but allows user to find the optimal range quicker than by manual trial and error.

In [3]:
tts = TTscan(constant = Quantity(10,'keV'), scan=100, polarization = 'p')

print(tts)

Scan type     : angle
Scan constant : 10 keV
Polarization  : pi
Scan points   : 100
Scan range    : automatic

Integrator                 : zvode_bdf
(Minimum) integration step : 1e-10 um
Alternative starting depth : None



### Solver parameters

TTscan also contains optional parameter for the integrator. Currently (v.1.0) PyTTE uses only the variable-coeffient ordinary differential equation solver (VODE) with backward
differential formula (BDF) method. However, it is possible to adjust the minimum intergration step of the VODE integrator which may speed up the integration but lead to numerical instability when the diffraction condition is not met in the crystal. 

In [4]:
tts = TTscan(constant = Quantity(10,'keV'), scan=100, polarization = 'p',solver = 'zvode_bdf', integration_step = Quantity(0.001, 'um'))

print(tts)

Scan type     : angle
Scan constant : 10 keV
Polarization  : pi
Scan points   : 100
Scan range    : automatic

Integrator                 : zvode_bdf
(Minimum) integration step : 0.001 um
Alternative starting depth : None



For thick crystals with a strain gradient, the integration will take a long time and numerical instability will eventually occur. If the diffraction takes place in the upper part of the crystal, the performance of the integration can be improved by starting the integration somewhere inside the crystal. This can be controlled with `start_depth`, which should be a value in between 0 (the top) and -thickness (the bottom of the crystal).

In [5]:
tts = TTscan(constant = Quantity(10,'keV'), scan=100, polarization = 'p', start_depth = Quantity(-100, 'um'))

print(tts)

Scan type     : angle
Scan constant : 10 keV
Polarization  : pi
Scan points   : 100
Scan range    : automatic

Integrator                 : zvode_bdf
(Minimum) integration step : 1e-10 um
Alternative starting depth : -100 um



## Initialization from a file

TTscan parameters can be written in a file, the path of which is passed as an argument to the constructor.

In [10]:
'''
Contents of TTscan_init.inp:

constant 12 keV
scan -50 50 100 arcsec
polarization sigma
solver zvode_bdf
integration_step 1 nm
'''

print(TTscan(filepath = 'TTscan_init.inp'))

Scan type     : angle
Scan constant : 12.0 keV
Polarization  : sigma
Scan points   : 100
Scan range    : manual from -50.0 to 50.0 arcsec

Integrator                 : zvode_bdf
(Minimum) integration step : 1.0 nm
Alternative starting depth : -0.1 mm

